In [ ]:
# default_exp error.torch

# Pytorch Errors

> All the possible errors that fastdebug can support and verbosify involving Pytorch

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#export
import torch
import re

## Errors

While some errrors are specifically designed for the [fastai](https://docs.fast.ai) library, the general idea still holds true in raw `Pytorch` as well. 

In [ ]:
#export
def device_error(e:Exception, a:str, b:str) -> Exception:
    """
    Verbose error for if `a` and `b` are on different devices
    Should be used when checking if a model is on the same device, or two tensors
    """
    inp, weight, _ = e.args[0].replace('( ', '').split(')')
    inp = inp.replace('Input type', f'{a} has type: \t\t')
    weight = weight.replace(' and weight type', f'{b} have type: \t')
    err = f'Mismatch between weight types\n\n{inp})\n{weight})\n\nBoth should be the same.'
    e.args = [err]
    raise e

The device error provides a much more readable error when `a` and `b` were on two different devices. An situation is below:
```python
inp = torch.rand().cuda()
model = model.cpu()
try:
    _ = model(inp)
except Exception as e:
    device_error(e, 'Input type', 'Model weights')
```
And our new log:
```bash
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-28-981e0ace9c38> in <module>()
      2     model(x)
      3 except Exception as e:
----> 4     device_error(e, 'Input type', 'Model weights')

10 frames
/usr/local/lib/python3.7/dist-packages/torch/tensor.py in __torch_function__(cls, func, types, args, kwargs)
    993 
    994         with _C.DisableTorchFunction():
--> 995             ret = func(*args, **kwargs)
    996             return _convert(ret, cls)
    997 

RuntimeError: Mismatch between weight types

Input type has type: 		 (torch.cuda.FloatTensor)
Model weights have type: 	 (torch.FloatTensor)

Both should be the same.
```

In [ ]:
#export
def get_shape(o:str) -> torch.Size:
    """
    Finds the size of a string representation of a torch layer, and returns it
    """
    m = re.findall(r'\[([0-9, +]+)\]', o)[0]
    shp = [int(i) for i in m.split(', ')]
    return torch.Size(shp)

All size errors in torch return a pattern of `expected [a, b, c] but got [d, e, f]`. `get_sz` only cares about that first part as we only want the size of the layer:

In [ ]:
size = get_sz("Model expected [2, 2]")
size

torch.Size([2, 2])

In [ ]:
#hide
test_eq(size, torch.Size((2,2)))

In [ ]:
#export
def get_layer_by_shape(m, weight_shape:torch.Size):
    "Grabs a layer in `m` of the same `shape`"
    for i, layer in enumerate(m.modules()):
        if hasattr(layer, 'weight'): 
            if layer.weight.shape == weight_shape:
                return i, layer

Returns the layer number and the actual torch layer in some model `m` so long as the shape matches.
An example is below:

In [ ]:
m = nn.Sequential(
    nn.Conv2d(3,3,1),
    nn.ReLU(),
    nn.Linear(3,2)
)

In [ ]:
get_layer_by_shape(m, torch.Size([2,3]))

(3, Linear(in_features=3, out_features=2, bias=True))

> Note: Currently this does not account for if multiple layers are in the same model of the same shape. Ideally we'd use Hooks instead but wasn't able to get this working. 

In [ ]:
#export
def layer_error(e:Exception, model) -> Exception:
    """
    Verbose error for when there is a size mismatch between some input and the model. `model` should be any torch model
    """
    args = e.args[0].replace("Expected", "Model expected")
    shape = get_shape(args)
    i, layer = get_layer_by_shape(model, shape)
    e.args = [f'Size mismatch between input tensors and what the model expects\n\n{args}\n\tat layer {i}: {layer}']
    raise e

`layer_error` can be used anywhere that you want to check that the inputs are right for some model.

Let's use our `m` model from earlier to show an example:

In [ ]:
inp = torch.rand(5,2, 3)
try:
    m(inp)
except Exception as e:
    layer_error(e, m)

RuntimeError: Size mismatch between input tensors and what the model expects

Model expected 4-dimensional input for 4-dimensional weight [3, 3, 1, 1], but got 3-dimensional input of size [5, 2, 3] instead
	at layer 1: Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))

Much more readable!